In [34]:
%reset -f

import gc
gc.collect()  # 強制 Python 回收記憶體

0

In [35]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [36]:
def process_csv_files(folder_path):
    # 使用 glob 取得資料夾內所有 CSV 檔案路徑
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

    if not csv_files:
        print("指定資料夾內沒有找到 CSV 檔案。")
        return

    for file in csv_files:
        try:
            # 讀取 CSV 檔案，假設檔案有標題列
            df = pd.read_csv(file)

            B = df['B DATA(T)']
            H = df['H DATA(A/m)']

            # print(B)
            # print(H)

            # 計算平均值
            avg_H = H.mean()
            avg_B = B.mean()

            print(
                f"檔案 {os.path.basename(file)} 的平均 H: {avg_H:.6f}，平均 B: {avg_B:.6f}"
            )

            # # 採樣後數據（從 get_dataloader 中提取）
            seq_length = 1024
            cols = range(0, 8192, int(8192 / seq_length))  # 下採樣到 128 點

            B_downsampled = B[cols]
            H_downsampled = H[cols]

            # 採樣後的 B-H 面積和 Pcv 計算
            # 確保採樣後數據形成閉合迴圈
            B_downsampled_closed = np.append(B_downsampled, B_downsampled[0])
            H_downsampled_closed = np.append(H_downsampled, H_downsampled[0])

            avg_H_128 = H_downsampled_closed.mean()
            avg_B_128 = B_downsampled_closed.mean()

            print(
                f"檔案 {os.path.basename(file)} 降階128後的平均 H: {avg_H_128:.6f}，平均 B: {avg_B_128:.6f}"
            )

            # # 繪製 B
            # plt.figure()
            # plt.plot(B)
            # plt.title(f"B - {os.path.basename(file)}")
            # plt.xlabel("Data point")
            # plt.ylabel("value")
            # plt.grid()
            # plt.show()

            # # 繪製 H
            # plt.figure()
            # plt.plot(H)
            # plt.title(f"H - {os.path.basename(file)}")
            # plt.xlabel("Data point")
            # plt.ylabel("value")
            # plt.grid()
            # plt.show()
            #
            # # 繪製 BH 曲線圖 (H 為 x 軸，B 為 y 軸)
            # plt.figure()
            # plt.plot(H, B, linestyle='-')
            # plt.title(f"BH Curve - {os.path.basename(file)}")
            # plt.xlabel("H")
            # plt.ylabel("B")
            # plt.grid()
            # plt.show()

        except Exception as e:
            print(f"讀取或處理 {file} 時發生錯誤：{e}")


if __name__ == '__main__':
    # 指定包含 CSV 檔案的資料夾路徑
    folder_path = r'./waveforms'  # 請依實際資料夾位置調整
    process_csv_files(folder_path)


檔案 CH467160_DC_TRI_100k_100mT_N10-10_Norm..csv 的平均 H: -0.000000，平均 B: 0.000000
檔案 CH467160_DC_TRI_100k_100mT_N10-10_Norm..csv 降階128後的平均 H: 0.000095，平均 B: -0.000001
檔案 CH467160_DC_TRI_100k_100mT_N10-11_Norm..csv 的平均 H: -0.000000，平均 B: 0.000000
檔案 CH467160_DC_TRI_100k_100mT_N10-11_Norm..csv 降階128後的平均 H: 0.000211，平均 B: -0.000001
檔案 CH467160_DC_TRI_100k_100mT_N10-2_Norm..csv 的平均 H: -0.000000，平均 B: -0.000000
檔案 CH467160_DC_TRI_100k_100mT_N10-2_Norm..csv 降階128後的平均 H: 0.000640，平均 B: -0.000001
檔案 CH467160_DC_TRI_100k_100mT_N10-3_Norm..csv 的平均 H: -0.000000，平均 B: 0.000000
檔案 CH467160_DC_TRI_100k_100mT_N10-3_Norm..csv 降階128後的平均 H: 0.000482，平均 B: -0.000001
檔案 CH467160_DC_TRI_100k_100mT_N10-4_Norm..csv 的平均 H: 0.000000，平均 B: -0.000000
檔案 CH467160_DC_TRI_100k_100mT_N10-4_Norm..csv 降階128後的平均 H: 0.000445，平均 B: -0.000001
檔案 CH467160_DC_TRI_100k_100mT_N10-5_Norm..csv 的平均 H: 0.000000，平均 B: 0.000000
檔案 CH467160_DC_TRI_100k_100mT_N10-5_Norm..csv 降階128後的平均 H: 0.000333，平均 B: -0.000001
檔案 CH467160_DC_TRI_100k_

In [37]:
def peak_aware_downsample(B_array, total_points=128, peak_ratio=0.3):
    """
    根據 B 波形正負峰值採樣密集區，其餘均勻取樣
    - B_array: shape=(8192,) 的 B 資料
    - total_points: 要保留的總點數（例如 128）
    - peak_ratio: 有多少比例點數保留給 B 高峰區段
    """
    assert len(B_array) >= 8192, "B 資料長度不足"

    peak_points = int(total_points * peak_ratio)
    edge_points = total_points - peak_points

    # 找出 B 的正負最大區段（峰值 ± buffer 區）
    idx_max = np.argmax(B_array)
    idx_min = np.argmin(B_array)

    # 避免極端集中：對 max/min 附近 ±Δ點取樣
    peak_width = 100  # 峰值附近取樣範圍（可調整）
    peak_indices = np.concatenate([
        np.linspace(max(0, idx_min - peak_width),
                    min(8191, idx_min + peak_width),
                    peak_points // 2,
                    dtype=int),
        np.linspace(max(0, idx_max - peak_width),
                    min(8191, idx_max + peak_width),
                    peak_points // 2,
                    dtype=int),
    ])

    # 其他部分均勻分布取樣
    exclude = set(peak_indices)
    linear_indices = np.linspace(0, 8191, edge_points * 2)  # 乘 2 是因為後面會篩掉重複
    linear_indices = [int(i) for i in linear_indices if int(i) not in exclude]
    linear_indices = np.array(linear_indices[:edge_points])

    indices = np.sort(np.unique(np.concatenate([peak_indices,
                                                linear_indices])))
    return indices
